# Introduction

ReviseMate is an end-to-end system that produces questions and answers based on input text or document. The perfect use case is for students during the examination week. They provide a document of a certain topic and test their knowledge against the questions produced by the system. It includes a simple and neat interface to answer the quiz, built using Streamlit. The models are tested against reference answers produced by a Gemini-pro which is a Large Language Model. The testing framework for different types of questions are custom built. They quantify the correctness of the answers produced by the models, and the best performing Small Language model is Mistral-7B.
Note: All the models are quantized to adjust to the memory constraints.

## Observations
- GPTQ models are superior to GGUF models in accuracy and performance
- mistral-7b-gguf > zephyr-7b-gguf
- If the quatization/calibration dataset for the gptq model is different from our intended usecase, then the gguf model will most likely give better performance than the GPTQ model. Because using the calibration dataset, the length of weights(fp-16, fp-8) of the model are decided.
- Sometimes GPTQ models act funny like giving responses to simple prompts but not slightly complex ones.
- Q4_K_M provides the best and most stable results among all the quantized Mistral models including the Q5_K_M which is supposedly the most effective and efficient.
- GGUF models can give close performace to its GPTQ counterpart with bigger GPUs and more layers offloaded to GPU.




## Libraries

In [1]:
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelera/te.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U langchain
!pip install -q -U ctransformers[cuda]
!pip install sentence-transformers
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install pypdf

!pip install ninja
!pip install fastparquet
!pip install torch>=2.1.0
!pip install safetensors>=0.3.2
!pip install sentencepiece>=0.1.97
!pip install pygments
!pip install websockets
!pip install regex
!pip install chromadb
!pip install --upgrade --quiet  langchain-google-genai pillow
!pip install -U langchain-community
!pip install streamlit

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/huggingface/accelera/te.git /tmp/pip-req-build-26oodwvs did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/huggingface/accelera/te.git /tmp/pip-req-build-26oodwvs did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 MB 6.1 MB/s eta 0:00

## Imports

In [2]:
import time
import re
import json
import math
import random
import re
import numpy as np
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.llms import CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import LLMChain, RetrievalQA
import torch
from accelerate import Accelerator
from abc import ABC, abstractmethod
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from chromadb.errors import InvalidDimensionException

import locale
locale.getpreferredencoding = lambda: "UTF-8"

## Constants

In [3]:
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
PATH="./input.txt"

## Document split

In [5]:
# Split input document into langchain document objects

loader = TextLoader(PATH)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = text_splitter.split_documents(docs)

In [ ]:
type(docs)

In [10]:
text = '''
text = """
Barack Obama
Official portrait, 2012
44th President of the United States
In office
January 20, 2009 – January 20, 2017
Vice PresidentJoe Biden
Preceded by George W. Bush
Succeeded byDonald Trump
United States Senator
from Illinois
In office
January 3, 2005 – November 16, 2008
Preceded by Peter Fitzgerald
Succeeded byRoland Burris
Member of the Illinois Senate
from the 13th district
In office
January 8, 1997 – November 4, 2004
Preceded by Alice Palmer
Succeeded byKwame Raoul
Personal detailsBarack Obama
Barack Hussein Obama II (/b ə ˈr ɑ ːk hu ː ˈse ɪn
o ʊ ˈb ɑ ːm ə/ ⓘ b ə-RAHK hoo- SAYN oh-BAH-m ə;[1]
born Augus t 4, 1961)  is an American politician who
served as the 44th president of the United States from
2009 to 2017. A member of the Democratic Party, he was
the first African-American president in U.S. history.
Obama previously served as a U.S. senator representing
Illinois from 2005 to 2008, as an Illinois state senator
from 1997  to 2004, and as a civil rights lawyer and
university lecturer.
Obama was born in Honolulu, Hawaii. He graduated
from Columbia University in 1983 with a B.A. in
political science and later worked as a community
organizer in Chicago. In 1988, Obama enrolled in
Harvard Law School, where he was the first black
president of the Harvard Law Review. He became a civil
rights attorney and an academic, teaching constitutional
law at the University of Chicago Law School from 1992
to 2004. He also went into elective politics. Obama
represented the 13th district in the Illinois Senate from
1997 until 2004, when he successfully ran for the U.S.
Senate. In 2008, after a close primary campaign against
Hillary Clinton, he was nominated by the Democratic
Party for president and chose Joe Biden as his running
mate. Obama was elected president, defeating
Republican Party nominee John McCain in the
presidential election and was inaugurated on January 20,
2009. Nine months later he was named the 2009 Nobel
Peace Prize laureate, a decision that drew a mixture of
praise and criticism.
Obama's first-term actions addressed the global financial
crisis, and included a major stimulus package, to guide
the econom y in recovering from the Great Recession, a
partial extension of George W. Bush's tax cuts,
legislation to reform health care, a major financial
regulation reform bill, and the end of a major U.S.
military presence in Iraq. Obama also appointed Supreme
Court justices Sonia Sotomayor and Elena Kagan, the
former being the first Hispanic American on the Supreme
Court. He ordered the counterterrorism raid which killed
Osama bin Laden and downplayed Bush'sBorn Barack Hussein Obama II
August 4, 1961
Honolulu, Hawaii, U.S.
Citizenship United States of America
Political partyDemocratic
Spouse Michelle Robinson (m. 1992)
Children Malia · Sasha
Parents Barack Obama Sr.
Ann Dunham
Relatives Obama family
Residence Kalorama, Washington, D.C.
Alma mater Columbia University (BA)
Harvard University (JD)
Occupation Politician · lawyer · author
Awards Full list
Signature
Website Official website (https://bara
ckobama.com)
Obama Foundation (https://
www.obama.org)
White House Archives (http
s://obamawhitehouse.archiv
es.gov)
Barack Obama's voice
Obama on the death of Osama bin Laden
Recorded May 2, 2011counterinsurgency model, expanding air strikes and
making extensive use of special forces, while
encouraging greater reliance on host-gove rnment
militaries. Obama also ordered military involvement in
Libya in order to implement UN Security Council
Resolution 1973 , contributing to the overthrow of
Muammar Gaddafi.
After winning re-election by defeating Republican
oppone nt Mitt Romney, Obama was sworn in for a
second term on January 20, 2013. In his second term,
Obama took steps to combat climate change, signing a
major international climate agreement and an executive
order to limit carbon emissions. Obama also presided
over the implementation of the Affordable Care Act and
other legislation passed in his first term, and he
negotiated a nuclear agreement with Iran and normalized
relations with Cuba. The number of American soldiers in
Afghanistan fell dramatically during Obama's second
term, though U.S. soldiers remained in the country
throughout  Obama's presidency. Obama promoted
inclusion for LGBT Americans, culminating in the
Supreme Court's decision to strike down same-sex
marriage bans as unconstitutional in Obergefell v.
Hodge s.
Obama left office on January 20, 2017, and continues to
reside in Washington, D.C. His presidential library in
Chicago began construction in 2021.  Since leaving
office, Obama has remained active in Democratic
politics, including campaigning for candidates in various
American elections, such as his former vice president Joe
Biden in his successful bid for president in 2020 . Outside
of politics, Obama has published three bestselling books :
Dreams from My Father (1995) , The Audac ity of Hope
(2006) , and A Promised Land (2020) . Rankings by
scholars and historians, in which he has been featured
since 2010, pl ace him in the middle to uppe r tier of American presidents.[2][3][4]
Obama was born on August 4, 1961,[5] at Kapiolani Medical Center for Women and Children in Honolulu,
Hawaii.[6][7][8][9] He is the only president born outside the contiguous  48 states.[10] He was born to an
American mother and a Kenyan father. His mother, Ann Dunha m (1942–1995) , was born in Wichita,
Kansas and was of English, Welsh, German, Swiss, and Irish descent. In 2007 it was discovered her great-
great-grandfather Falmouth Kearney emigrated from the village of Moneygall, Ireland to the US in
1850.[11] In July 2012, Ancestry.com found a strong likelihood that Dunha m was descended from John
Punch, an enslaved African man who lived in the Colony of Virginia during the seventeenth century.[12][13]
Obama's father, Barack Obama Sr. (1934–19 82),[14][15] was a married[16][17][18] Luo Kenyan from
Nyang'oma Kogelo.[16][19] His last name, Obama, was derived from his Luo descent.[20] Obama's parents
0:000:00
/ 0:00/ 0:00
Early life and careerObama (right) with grandfather
Stanley Armour Dunham, mother Ann
Dunham, and half-sister Maya
Soetoro, mid-1970s in Honolulumet in 1960 in a Russian langua ge class at the University of
Hawai ʻ i at Mānoa, where his father was a foreign student on a
scholarship.[21][22] The couple married in Wailuku, Hawaii, on
February 2, 1961, s ix months before Obama was born.[23][24]
In late August 1961,  a few weeks after he was born, Barack and his
mother moved to the University of Washington in Seattle, where
they lived for a year. During that time, Barack's father completed
his unde rgraduate degree in econom ics in Hawaii, graduating in
June 1962. He left to attend graduate school on a scholarship at
Harvard University, where he earned an M.A. in econom ics.
Obama's parents divorced in March 1964.[25] Obama Sr. returned
to Kenya in 1964, where he married for a third time and worked for
the Kenyan gove rnment as the Senior Econom ic Analyst in the
Ministry of Finance.[26] He visited his son in Hawaii only once, at Christmas 1971,[27] before he was killed
in an automobile accident in 1982, when Obama was 21 years old.[28] Recalling his early childhood,
Obama said: "That my father looke d nothing like the people around me—that he was black as pitch, my
mother white as milk—ba rely registered in my mind."[22] He described his struggles as a young adult to
reconcile social perceptions of his multiracial heritage.[29]
In 1963, Dunha m met Lolo Soetoro at the University of Hawaii; he was an Indone sian East–West Center
graduate student in geography. The couple married on Molokai on March 15, 1965.[30] After two one-year
extensions of his J-1 visa, Lolo returned to Indone sia in 1966. His wife and stepson followed sixteen
months later in 1967 . The family initially lived in the Menteng Dalam neighbor hood in the Tebet district of
South Jakarta. From 1970, they lived in a wealthier neighbor hood in the Menteng district of Central
Jakarta.[31]
At the age of six, Obama and his mother had moved to Indone sia to join his stepfather. From age six to ten,
he was registered in school as "Barry"[32] and attended local Indone sian-langua ge schools: Sekolah Dasar
Katolik Santo Fransiskus Asisi (St. Francis of Assisi Catholic Elementary School) for two years and
Sekolah Dasar Negeri Menteng 01 (State Elementary School Menteng 01) for one and a half years,
supplemented by English-langua ge Calvert School homeschooling by his mother.[33][34] As a result of his
four years in Jakarta, he was able to speak Indone sian fluently as a child.[35] During his time in Indone sia,
Obama's stepfather taught him to be resilient and gave him "a pretty hardheaded assessment of how the
world works".[36]
In 1971, Obama returned to Honolulu to live with his maternal grandpa rents, Madelyn and Stanley
Dunha m. He attended Punahou School—a private college preparatory school—with the aid of a
scholarship from fifth grade until he graduated from high school in 1979.[37] In high school, Obama
continued to use the nickname "Barry" which he kept until making a visit to Kenya in 1980.[38] Obama
lived with his mother and half-sister, Maya Soetoro, in Hawaii for three years from 1972 to 1975 while his
mother was a graduate student in anthropology at the University of Hawaii.[39] Obama chose to stay in
Hawaii when his mother and half-sister returned to Indone sia in 1975, so his mother could begin
anthropology field work.[40] His mother spent most of the next two decades in Indone sia, divorcing Lolo
Soetoro in 1980 and earning a PhD degree in 1992, before dying in 1995 in Hawaii following unsuccessful
treatment for ovarian and uterine cancer.[41]EducationObama's Indonesian school record in
St. Francis of Assisi Catholic
Elementary School. Obama was
enrolled as "Barry Soetoro" (no. 1),
and was wrongly recorded as an
Indonesian citizen (no. 3) and a
Muslim (no. 4).[32]
External videos
 Derrick Bell threatens to leave
Harvard (http://bostonlocaltv.org/cat
alog/V_UDAMVZGA4JEY06N), April
24, 1990, 11:34, Boston TV Digital
Archive[60] Student Barack Obama
introduces Professor Derrick Bell
starting at 6:25.Of his years in Honolulu, Obama wrote: "The oppor tunity that
Hawaii offered — to experience a variety of cultures in a climate of
mutual respect — became an integral part of my world view, and a
basis for the values that I hold most dear."[42] Obama has also
written and talked about using alcohol, marijuana, and cocaine
during his teenage years to "push questions of who I was out of my
mind".[43] Obama was also a member of the "Choom Gang" (the
slang term for smoking marijuana), a self-named group of friends
who spent time together and smoked marijuana.[44][45]
After graduating from high school in 1979, Obama moved to Los
Angeles to attend Occidental College on a full scholarship. In
February 1981, Obama made his first public speech, calling for
Occidental to participate in the disinvestment from South Africa in
respons e to that nation's policy of apartheid.[46] In mid-1981,
Obama traveled to Indone sia to visit his mother and half-sister
Maya, and visited the families of college friends in Pakistan for
three weeks.[46] Later in 1981, he transferred to Columbia
University in New York City as a junior, where he majored in
political science with a specialty in international relations[47] and in
English literature[48] and lived off-campus on West 109th Street.[49]
He graduated with a Bachelor of Arts degree in 1983 and a 3.7 GPA. After graduating, Obama worked for
about a year at the Business International Corporation, where he was a financial researcher and
writer,[50][51] then as a project coordinator for the New York Public Interest Research Group on the City
College of New York campus for three months in 1985.[52][53][54]
Two years after graduating from Columbia, Obama moved from New York to Chicago when he was hired
as director of the Developing Communities Project, a faith-based community organization originally
comprising eight Catholic parishes in Roseland, West Pullman, and Riverdale on Chicago's South Side. He
worked there as a community organizer from June 1985 to May 1988.[53][55] He helped set up a job
training program, a college preparatory tutoring program, and a tenants' rights organization in Altgeld
Gardens.[56] Obama also worked as a consultant and instructor for the Gamaliel Founda tion, a community
organizing institute.[57] In mid-1988, he traveled for the first time in Europe for three weeks and then for
five weeks in Kenya, where he met many of  his paternal relatives for the first time.[58][59]
Despite being offered a full scholarship to Northwestern
University School of Law, Obama enrolled at Harvard Law
School in the fall of 1988, living in nearby Somerville,
Massachusetts.[61] He was selected as an editor of the Harvard
Law Review at the end of his first year,[62] president of the journal
in his second year,[56][63] and research assistant to the
constitutional scholar Laurence Tribe while at Harvard.[64]
During his summers, he returned to Chicago, where he worked as
a summer associate at the law firms of Sidley Austin in 1989 and
Hopkins & Sutter in 1990.[65] Obama's election as the first black
president of the Harvard Law Review gained national mediaCollege and research jobs
Community organizer and Harvard Law SchoolObama poses in the Green Room of
the White House with wife Michelle
and daughters Sasha and Malia,
2009.attention[56][63] and led to a publishing contract and advance for a book about race relations,[66] which
evolved into a personal memoir. The manuscript was publ ished in mid-1995 a s Dreams from My Father.[66]
Obama graduated from Harvard Law in 1991 w ith a Juris Doctor magna c um laude .[67][62]
In 1991, Obama accepted a two-year position as Visiting Law and Government Fellow at the University of
Chicago Law School to work on his first book.[66][68] He then taught constitutional law at the University of
Chicago Law School for twelve years, first as a lecturer from 1992 to 1996, and then as a senior lecturer
from 1996 t o 2004.[69]
From April to October 1992, Obama directed Illinois's Project Vote, a voter registration campaign with ten
staffers and seven hundred volunteer registrars; it achieved its goal of registering 150,000 of 400,000
unregistered African Americans in the state, leading Crain's Chicago Business to name Obama to its 1993
list of "40 unde r Forty" pow ers to be.[70]
In a 2006 interview, Obama highlighted the diversity of his extended family: "It's like a little mini-United
Nations," he said. "I've got relatives who look like Bernie Mac, and I've got relatives who look like
Margaret Thatcher."[71] Obama has a half-sister with whom he was raised (Maya Soetoro-Ng) and seven
other half-siblings from his Kenyan father's family, six of them living.[72] Obama's mother was survived by
her Kansas-born mother, Madelyn Dunham,[73] until her death on November 2, 2008,[74] two days before
his election to the presidency. Obama also has roots in Ireland; he met with his Irish cousins in Moneygall in
May 2011.[75] In Dreams from My Father, Obama ties his mother's family history to possible Native
American ancestors and distant relatives of Jefferson Davis, President of the Confederate States of America
during the American Civil War. He also shares distant ancestors in common with George W. Bush and Dick
Cheney, among ot hers.[76][77][78]
Obama lived with anthropologist Sheila Miyoshi Jager while he was a community organizer in Chicago in
the 1980s .[79] He proposed to her twice, but both Jager and her parents turned him down.[79][80] The
relationship was not made publ ic until May 2017, s everal months after his presidency had ended.[80]
In June 1989, Obama met Michelle Robinson when he was
employed Sidley Austin.
"""'''

### If we want to split the input text based on succeding paragraphs


In [ ]:
def split_text(text: str):
    """
    Splits a text string into a list of non-empty substrings based on the specified pattern.
    The "\n \n" pattern will split the document para by para
    Parameters:
    - text (str): The input text to be split.

    Returns:
    - List[str]: A list containing non-empty substrings obtained by splitting the input text.

    """
    split_text = re.split('\n \n', text)
    return [i for i in split_text if i != ""]

chunked_text = split_text(text=text)

## Initialize models

### Mistral-7b - download and load

In [4]:
# This is the Q4_K_M (4-bit) quantization that performs the best
!huggingface-cli download TheBloke/OpenHermes-2.5-Mistral-7B-16k-GGUF openhermes-2.5-mistral-7b-16k.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
(…)penhermes-2.5-mistral-7b-16k.Q4_K_M.gguf: 100% 4.37G/4.37G [03:30<00:00, 20.7MB/s]
Download complete. Moving file to openhermes-2.5-mistral-7b-16k.Q4_K_M.gguf
openhermes-2.5-mistral-7b-16k.Q4_K_M.gguf


In [5]:
accelerator = Accelerator()

config = {'max_new_tokens': 10000, 'repetition_penalty': 1.1, 'context_length': 16000, 'temperature':0, 'gpu_layers': 50}
llm = CTransformers(model = "./openhermes-2.5-mistral-7b-16k.Q4_K_M.gguf", model_type = "mistral", gpu_layers=50, config=config, mlock=True)

llm, config = accelerator.prepare(llm, config)

print("LLM Initialized...")

LLM Initialized...


In [13]:
llm

CTransformers(client=<ctransformers.llm.LLM object at 0x7a237e834dc0>, model='./openhermes-2.5-mistral-7b-16k.Q4_K_M.gguf', model_type='mistral', config={'max_new_tokens': 10000, 'repetition_penalty': 1.1, 'context_length': 16000, 'temperature': 0, 'gpu_layers': 50})

In [ ]:
# This is the Q5_K_M (5-bit) quantization that supposed to outperform Q4_K_M. I found it otherwise

!huggingface-cli download TheBloke/OpenHermes-2.5-Mistral-7B-16k-GGUF openhermes-2.5-mistral-7b-16k.Q5_K_M.gguf --local-dir . --local-dir-use-symlinks False

accelerator = Accelerator()

config = {'max_new_tokens': 10000, 'repetition_penalty': 1.1, 'context_length': 16000, 'temperature':0, 'gpu_layers': 50}
llm = CTransformers(model = "./openhermes-2.5-mistral-7b-16k.Q5_K_M.gguf", model_type = "mistral", gpu_layers=50, config=config, mlock=True)

llm, config = accelerator.prepare(llm, config)

print("LLM Initialized...")

### Mistral-7b-Instruct - download and load

In [ ]:
# Q4_K_M (4-bit) version
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF mistral-7b-instruct-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

In [ ]:
accelerator = Accelerator()

config = {'max_new_tokens': 50000, 'repetition_penalty': 1.1, 'context_length': 16000, 'temperature':0, 'gpu_layers': 50}
llm = CTransformers(model = "./mistral-7b-instruct-v0.1.Q4_K_M.gguf", model_type = "mistral", gpu_layers=50, config=config, mlock=True)

llm, config = accelerator.prepare(llm, config)

print("LLM Initialized...")

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmp319ou11m
mistral-7b-instruct-v0.1.Q4_K_M.gguf: 100% 4.37G/4.37G [00:47<00:00, 92.9MB/s]
./mistral-7b-instruct-v0.1.Q4_K_M.gguf


### Zephyr - download and load


In [ ]:
!huggingface-cli download TheBloke/zephyr-7B-beta-GGUF zephyr-7b-beta.Q5_K_M.gguf --local-dir . --local-dir-use-symlinks False

In [ ]:
accelerator = Accelerator()

config = {'max_new_tokens': 50000, 'repetition_penalty': 1.1, 'context_length': 16000, 'temperature':0, 'gpu_layers': 50}
llm = CTransformers(model = "./zephyr-7b-beta.Q5_K_M.gguf", model_type = "mistral", gpu_layers=50, config=config, mlock=True)

llm, config = accelerator.prepare(llm, config)

print("LLM Initialized...")

### Gemini-pro - Load

In [8]:
import getpass
import os

In [14]:
# llm_google = ChatGoogleGenerativeAI(model="gemini-pro")
# llm_google

## QuizGenerator class

In [7]:
import re
import json
import math
import time
import random
import logging
from abc import ABC, abstractmethod
from typing import List, Dict, Optional, Any, Tuple
from langchain_core.runnables import RunnableLambda

# logging.basicConfig(level=logging.INFO)

class QuizGenerator(ABC):
    """
    Abstract class for generating quiz questions.
    """

    def __init__(self, chunk_size: int, chunk_overlap: int, llm: Any, number_of_questions: int, difficulty: str, question_type: str):
        self.llm = llm
        self.number_of_questions = number_of_questions
        self.difficulty = difficulty
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.question_type = question_type

    @abstractmethod
    def generate_template(self) -> str:
        """
        Generate a template for the LLM prompt.
        """
        pass

    @abstractmethod
    def generate_template_and_format(self) -> Tuple[str, str]:
        """
        Generate a template and format instructions for the LLM prompt.
        """
        pass

    def text_splitter(self, data: str) -> List[str]:
        """
        Splits the input text into chunks based on specified chunk size and overlap.
        """
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap, add_start_index=True
        )
        all_splits = text_splitter.split_text(data)
        return all_splits

    def file_processing(self, loader: Any) -> List[str]:
        """
        Processes the input file using the provided loader.
        """
        data = loader.load()
        text = ''.join(page.page_content for page in data)
        return self.text_splitter(data=text)

    def parse_json_like(self, text: str) -> List[Dict[str, str]]:
        """
        Parses JSON-like strings from the input text.
        """
        object_pattern = re.compile(r'\{.*?\}', re.DOTALL)
        pair_pattern = re.compile(r'"(.*?)":\s*"(.*?)"')

        objects = object_pattern.findall(text)
        dict_list = []

        for obj in objects:
            pairs = pair_pattern.findall(obj)
            d = {key: value for key, value in pairs}
            dict_list.append(d)

        return dict_list

    def create_llm_chain(self) -> RunnableLambda:
        """
        Creates a LLM chain using the generated template and format instructions.
        """
        template_str, format_instructions = self.generate_template_and_format()
        template = PromptTemplate.from_template(template_str)
        print(f"Template: {template}, Type: {type(template)}")

        def format_template(inputs):
            inputs['format_instructions'] = format_instructions
            return template.format(**inputs)

        runnable_template = RunnableLambda(format_template)
        llm_chain = runnable_template | self.llm
        return llm_chain

    def preprocess_questions(self, questions, type):

      updated_questions = []

      for obj in questions:
          obj['type'] = type
          obj['answer'] = re.sub(r'[abcd]\) ', '', obj['answer'])
          updated_questions.append(obj)

      if type == "MCQ":
          updated_questions = [
      {**question, 'options': [question['a)'], question['b)'], question['c)'], question['d)']]}
      for question in updated_questions]

      return updated_questions

    def generate_qa(self, file_path: Optional[str] = None, text: Optional[str] = None) -> List[Dict[str, str]]:
        """
        Generates quiz questions based on the provided file or text data.
        """
        if file_path and text:
            print("Input either a file or text data. Not both.")
            return []

        if file_path:
            if file_path.endswith(".txt"):
                loader = TextLoader(file_path)
            elif file_path.endswith(".pdf"):
                loader = PyPDFLoader(file_path)
            else:
                print("Unsupported file type.")
                return []

            chunks = self.file_processing(loader)
        elif text:
            chunks = self.text_splitter(text)
        else:
            print("Please provide either a file path or text data.")
            return []

        self.number_of_questions_per_chunk = math.ceil(self.number_of_questions / len(chunks))
        print(f"Length of chunks: {len(chunks)}")

        llm_chain = self.create_llm_chain()
        print("LLM Chain created")

        start = time.time()
        results = []

        for chunk in chunks:
            quiz = llm_chain.invoke({'text': chunk})
            result = self.parse_json_like(quiz)
            print(f"Result length: {len(result)}")
            print(f"Result: {result}")
            results.extend(result)

        end = time.time()
        random.shuffle(results)
        results = results[:self.number_of_questions]

        results = self.preprocess_questions(results, self.question_type)

        with open(f'{self.question_type}.json', 'w', encoding='utf-8') as f:
            json.dump({"Total time taken": end - start}, f)
            json.dump(results, f, indent=2)

        return results

## Different question types

### Mulitple Choice Questions

In [8]:
class MCQ(QuizGenerator):
    """
    Concrete implementation of QuizGenerator for generating multiple-choice questions.
    """

    def __init__(self, chunk_size: int, chunk_overlap: int, llm: Any, number_of_questions: int, difficulty: str):
        super().__init__(chunk_size, chunk_overlap, llm, number_of_questions, difficulty, 'mcq')

    def generate_template(self) -> str:
        """
        Generate a template for creating multiple-choice questions.
        """
        template = f"""<s>[INST]Multiple-choice questions (MCQs) are a type of objective assessment item that presents respondents with a question or statement followed by a set of predefined answer options. The respondent must choose the correct answer from the provided options. Each question typically has only one correct answer.
                    You are an expert at creating scenario-based Multiple Choice Questions and answers, based on the given text and documentation.
                    Your goal is to make users acquainted with the content of the text through well-crafted Multiple Choice questions you generate.
                    IMPORTANT: Also, for each question, give the correct answer and don't use incomplete sentences as context.
                    IMPORTANT: Do not repeat the same question. Do not rephrase/jumble the same question multiple times.
                    You should also make sure to include the question and solution in one single post and not split them apart.
                    Make sure not to lose any important information.
                    Set the difficulty of the questions to {self.difficulty}. Make sure all questions are set to the specified difficulty.

                    The input text given is:

                    ------------
                    {{text}}
                    ------------

                    IMPORTANT: Create exactly {self.number_of_questions_per_chunk} questions and return the correct answer.
                    IMPORTANT: Do not create more than {self.number_of_questions_per_chunk} questions or less than {self.number_of_questions_per_chunk} questions.
                    IMPORTANT: For each question, give four options with one of them being the correct answer and the other options being incorrect (e.g., a) Incorrect, b) Incorrect, c) Incorrect, d) Correct, answer: d) Correct).
                    IMPORTANT: Make sure to follow the same format {{format_instructions}} for all questions.
                    Think clearly and follow the instructions tagged IMPORTANT with utmost criticality.
                    [/INST]</s>"""
        return template

    def generate_template_and_format(self) -> Tuple[str, str]:
        """
        Generate a template and format instructions for the LLM prompt.
        """
        template = self.generate_template()
        response_schema = [
            ResponseSchema(name="question", description="A multiple choice question generated from input text snippet."),
            ResponseSchema(name="a)", description="First option for the multiple choice question."),
            ResponseSchema(name="b)", description="Second option for the multiple choice question."),
            ResponseSchema(name="c)", description="Third option for the multiple choice question."),
            ResponseSchema(name="d)", description="Fourth option for the multiple choice question."),
            ResponseSchema(name="answer", description="Correct answer for the question. Use this format: 'd) the correct answer', etc.")
        ]

        output_parser = StructuredOutputParser.from_response_schemas(response_schema)
        format_instructions = output_parser.get_format_instructions()

        return template, format_instructions


### True or False questions

In [ ]:
class TrueFalse(QuizGenerator):
    def __init__(self, chunk_size: int, chunk_overlap: int, llm, number_of_questions: int, difficulty: str):
        """
        Initialize the TrueFalse class.

        Args:
            chunk_size (int): Size of the chunks for text splitting.
            chunk_overlap (int): Overlap size between chunks.
            llm (object): The large language model to be used.
            number_of_questions (int): Number of questions to generate.
            difficulty (str): Difficulty level of the questions.
        """
        super().__init__(chunk_size, chunk_overlap, llm, number_of_questions, difficulty, 'trueFalse')

    def generate_template(self) -> str:
        """
        Generate the prompt template for True/False questions.

        Returns:
            str: The prompt template string.
        """
        template = f""" <s>[INST]True/false questions are a type of binary or dichotomous closed-ended question format. In a true/false question, respondents are presented with a statement, and they are required to determine whether the statement is true or false. This format simplifies the response choices to two options, making it straightforward
                    You are an expert at creating scenario based True or False question and answers, based on the given text and documentation.
                    IMPORTANT: Your goal is to make users acquainted with the content of the text through well-crafted True/False questions you generate.
                    Also, for each question, give the correct answer and don't use incomplete sentences as context.
                    IMPORTANT: Do not repeat the same question. Do not rephrase/jumble the same question multiple times.
                    You should also make sure to include the question and solution in one single post and not split them apart.
                    Make sure not to lose any important information.
                    Set the difficulty of the questions to {self.difficulty}. Make sure all questions are set to the specified difficulty.

                    The input text given is:

                    ------------
                    {{text}}
                    ------------

                    Create {self.number_of_questions_per_chunk} questions and return the correct answer.
                    IMPORTANT: For each question, the correct answer must be binary, that is, either True or False only.
                    Make sure to follow the same format {{format_instructions}} for all questions.
                    Think clearly and follow the instructions tagged IMPORTANT with utmost criticality.
                    [/INST]</s>"""
        return template

    def generate_template_and_format(self) -> tuple:
        """
        Generate the prompt template and format instructions for True/False questions.

        Returns:
            tuple: A tuple containing the prompt template and format instructions.
        """
        template = self.generate_template()
        response_schema = [
            ResponseSchema(name="question", description="A True/False question generated from input text snippet."),
            ResponseSchema(name="answer", description="Correct answer for the question which is either 'True' or 'False' only.")
        ]

        output_parser = StructuredOutputParser.from_response_schemas(response_schema)
        format_instructions = output_parser.get_format_instructions()

        return template, format_instructions


### Open-ended questions

In [18]:
class OpenEnded(QuizGenerator):
    def __init__(self, chunk_size: int, chunk_overlap: int, llm, number_of_questions: int, difficulty: str):
        """
        Initialize the OpenEnded class.

        Args:
            chunk_size (int): Size of the chunks for text splitting.
            chunk_overlap (int): Overlap size between chunks.
            llm (object): The large language model to be used.
            number_of_questions (int): Number of questions to generate.
            difficulty (str): Difficulty level of the questions.
        """
        super().__init__(chunk_size, chunk_overlap, llm, number_of_questions, difficulty, 'openEnded')

    def generate_template(self) -> str:
        """
        Generate the prompt template for open-ended questions.

        Returns:
            str: The prompt template string.
        """
        template = f""" <s>[INST]Open-ended questions are questions that cannot be answered with a simple 'yes' or 'no' or with a brief piece of information. Instead, they require a more elaborate and detailed response, encouraging the respondent to express their thoughts, opinions, or knowledge on a given topic.
                    You are an expert at creating scenario based Open-ended Question and answers, based on the given text and documentation.
                    Your goal is to make users acquainted with the content of the text through well-crafted Open-ended questions you generate.
                    IMPORTANT: Also, for each question, give the correct answer and don't use incomplete sentences as context.
                    IMPORTANT: Do not repeat the same question. Do not rephrase/jumble the same question multiple times.
                    You should also make sure to include the question and solution in one single post and not split them apart.
                    Make sure not to lose any important information.
                    Set the difficulty of the questions to {self.difficulty}. Make sure all questions are set to the specified difficulty.

                    The input text given is:

                    ------------
                    {{text}}
                    ------------

                    Create {self.number_of_questions_per_chunk} questions and return the correct answer.
                    IMPORTANT: Generate questions which have to be answered with at least half a dozen words along with a clear, descriptive answer.
                    Make sure to follow the same format {{format_instructions}} for all questions.
                    Think clearly and follow the instructions tagged IMPORTANT with utmost criticality.
                    [/INST]</s>"""
        return template

    def generate_template_and_format(self) -> tuple:
        """
        Generate the prompt template and format instructions for open-ended questions.

        Returns:
            tuple: A tuple containing the prompt template and format instructions.
        """
        template = self.generate_template()
        response_schema = [
            ResponseSchema(name="question", description="An open-ended question generated from input text snippet."),
            ResponseSchema(name="answer", description="Detailed answer for the open-ended question.")
        ]

        output_parser = StructuredOutputParser.from_response_schemas(response_schema)
        format_instructions = output_parser.get_format_instructions()

        return template, format_instructions


### Fill in the Blanks

In [ ]:
class FillInTheBlanks(QuizGenerator):
    def __init__(self, chunk_size: int, chunk_overlap: int, llm, number_of_questions: int, difficulty: str):
        """
        Initialize the FillInTheBlanks class.

        Args:
            chunk_size (int): Size of the chunks for text splitting.
            chunk_overlap (int): Overlap size between chunks.
            llm (object): The large language model to be used.
            number_of_questions (int): Number of questions to generate.
            difficulty (str): Difficulty level of the questions.
        """
        super().__init__(chunk_size, chunk_overlap, llm, number_of_questions, difficulty, "fillInTheBlanks")

    def generate_template(self) -> str:
        """
        Generate the prompt template for fill-in-the-blanks questions.

        Returns:
            str: The prompt template string.
        """
        template = f""" <s>[INST]Fill in the blanks questions, also known as completion questions, are a type of assessment or educational exercise where individuals are provided with a sentence, paragraph, or passage with one or more missing words or phrases. The task is to fill in the missing elements to complete the text.
                    You are an expert at creating scenario based Fill in the blanks questions and answers, based on the given text and documentation.
                    Your goal is to make users acquainted with the content of the text through well-crafted Fill in the blanks questions you generate.
                    IMPORTANT: Also, for each question, give the correct answer and don't use incomplete sentences as context.
                    IMPORTANT: Do not repeat the same question. Do not rephrase/jumble the same question multiple times.
                    You should also make sure to include the question and solution in one single post and not split them apart.
                    Make sure not to lose any important information.
                    Set the difficulty of the questions to {self.difficulty}. Make sure all questions are set to the specified difficulty.

                    The input text given is:

                    ------------
                    {{text}}
                    ------------

                    Create {self.number_of_questions_per_chunk} questions and return the correct answer.
                    IMPORTANT: Generate statements with one keyword replaced by a '_____' which makes up the question. The answer is then the replaced keyword.
                    Make sure to follow the same format {{format_instructions}} for all questions.
                    Think clearly and follow the instructions tagged IMPORTANT with utmost criticality.
                    [/INST]</s>"""
        return template

    def generate_template_and_format(self) -> tuple:
        """
        Generate the prompt template and format instructions for fill-in-the-blanks questions.

        Returns:
            tuple: A tuple containing the prompt template and format instructions.
        """
        template = self.generate_template()
        response_schema = [
            ResponseSchema(name="question", description="A fill-in-the-blanks question with the keyword represented by a blank, generated from input text snippet."),
            ResponseSchema(name="answer", description="Correct answer for the question. Provide the answer with blanks filled.")
        ]

        output_parser = StructuredOutputParser.from_response_schemas(response_schema)
        format_instructions = output_parser.get_format_instructions()

        return template, format_instructions


### Mixed questions

In [10]:
class Mixed(QuizGenerator):
    def __init__(self, chunk_size: int, chunk_overlap: int, llm, number_of_questions: int, difficulty: str):
        """
        Initialize the Mixed class.

        Args:
            chunk_size (int): Size of the chunks for text splitting.
            chunk_overlap (int): Overlap size between chunks.
            llm (object): The large language model to be used.
            number_of_questions (int): Number of questions to generate.
            difficulty (str): Difficulty level of the questions.
        """
        super().__init__(chunk_size, chunk_overlap, llm, number_of_questions, difficulty, "mixed")
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.llm = llm
        self.number_of_questions = number_of_questions
        self.difficulty = difficulty

    def combined_questions(self, mcq, trueFalse, openEnded, text: str) -> list:
        """
        Generate combined questions from multiple types (MCQ, True/False, Open-Ended).

        Args:
            mcq (MCQ): An instance of the MCQ class.
            trueFalse (TrueFalse): An instance of the TrueFalse class.
            openEnded (OpenEnded): An instance of the OpenEnded class.
            text (str): The input text to generate questions from.

        Returns:
            list: A list of combined questions.
        """
        mcqs = mcq.generate_qa(text=text)
        tfs = trueFalse.generate_qa(text=text)
        ops = openEnded.generate_qa(text=text)
        return mcqs + ops + tfs

    def generate_qa(self, file_path: str = None, text: str = None) -> list:
        """
        Generate mixed questions from file or text input.

        Args:
            file_path (str, optional): Path to the input file. Defaults to None.
            text (str, optional): Input text data. Defaults to None.

        Returns:
            list: A list of generated questions.
        """
        if file_path and text:
            print("Input either a file or text data. Not both.")
            return []

        elif file_path:
            chunks = self.file_processing(file_path)

        elif text:
            chunks = self.text_splitter(text)

        else:
            print("Please provide either a file path or text data.")
            return []

        self.number_of_questions_per_chunk = math.ceil(self.number_of_questions / (len(chunks) * 3))
        mcq = MCQ(self.chunk_size, self.chunk_overlap, self.llm, self.number_of_questions_per_chunk, self.difficulty)
        trueFalse = TrueFalse(self.chunk_size, self.chunk_overlap, self.llm, self.number_of_questions_per_chunk, self.difficulty)
        openEnded = OpenEnded(self.chunk_size, self.chunk_overlap, self.llm, self.number_of_questions_per_chunk, self.difficulty)

        start = time.time()
        results = []
        for chunk in chunks:
            result = self.combined_questions(mcq, trueFalse, openEnded, chunk)
            print("result len: ", len(result))
            print(f"result: {result}")
            results.extend(result)
        end = time.time()
        random.shuffle(results)
        results = results[:self.number_of_questions]
        with open('mixed.json', 'w', encoding='utf-8') as f:
            json.dump(f"Total time taken: {end-start}", f)
            json.dump(results, f, indent=2)
        print(end - start)
        return results


## Combined cell with the parent class and all the child classes

In [ ]:
import re
import json
import math
import time
import random
import logging
from abc import ABC, abstractmethod
from typing import List, Dict, Optional, Any, Tuple
from langchain_core.runnables import RunnableLambda

# logging.basicConfig(level=logging.INFO)

class QuizGenerator(ABC):
    """
    Abstract class for generating quiz questions.
    """

    def __init__(self, chunk_size: int, chunk_overlap: int, llm: Any, number_of_questions: int, difficulty: str, question_type: str):
        self.llm = llm
        self.number_of_questions = number_of_questions
        self.difficulty = difficulty
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.question_type = question_type

    @abstractmethod
    def generate_template(self) -> str:
        """
        Generate a template for the LLM prompt.
        """
        pass

    @abstractmethod
    def generate_template_and_format(self) -> Tuple[str, str]:
        """
        Generate a template and format instructions for the LLM prompt.
        """
        pass

    def text_splitter(self, data: str) -> List[str]:
        """
        Splits the input text into chunks based on specified chunk size and overlap.
        """
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap, add_start_index=True
        )
        all_splits = text_splitter.split_text(data)
        return all_splits

    def file_processing(self, loader: Any) -> List[str]:
        """
        Processes the input file using the provided loader.
        """
        data = loader.load()
        text = ''.join(page.page_content for page in data)
        return self.text_splitter(data=text)

    def parse_json_like(self, text: str) -> List[Dict[str, str]]:
        """
        Parses JSON-like strings from the input text.
        """
        object_pattern = re.compile(r'\{.*?\}', re.DOTALL)
        pair_pattern = re.compile(r'"(.*?)":\s*"(.*?)"')

        objects = object_pattern.findall(text)
        dict_list = []

        for obj in objects:
            pairs = pair_pattern.findall(obj)
            d = {key: value for key, value in pairs}
            dict_list.append(d)

        return dict_list

    def create_llm_chain(self) -> RunnableLambda:
        """
        Creates a LLM chain using the generated template and format instructions.
        """
        template_str, format_instructions = self.generate_template_and_format()
        template = PromptTemplate.from_template(template_str)
        print(f"Template: {template}, Type: {type(template)}")

        def format_template(inputs):
            inputs['format_instructions'] = format_instructions
            return template.format(**inputs)

        runnable_template = RunnableLambda(format_template)
        llm_chain = runnable_template | self.llm
        return llm_chain

    def generate_qa(self, file_path: Optional[str] = None, text: Optional[str] = None) -> List[Dict[str, str]]:
        """
        Generates quiz questions based on the provided file or text data.
        """
        if file_path and text:
            print("Input either a file or text data. Not both.")
            return []

        if file_path:
            if file_path.endswith(".txt"):
                loader = TextLoader(file_path)
            elif file_path.endswith(".pdf"):
                loader = PyPDFLoader(file_path)
            else:
                print("Unsupported file type.")
                return []

            chunks = self.file_processing(loader)
        elif text:
            chunks = self.text_splitter(text)
        else:
            print("Please provide either a file path or text data.")
            return []

        self.number_of_questions_per_chunk = math.ceil(self.number_of_questions / len(chunks))
        print(f"Length of chunks: {len(chunks)}")

        llm_chain = self.create_llm_chain()
        print("LLM Chain created")

        start = time.time()
        results = []

        for chunk in chunks:
            quiz = llm_chain.invoke({'text': chunk})
            # print(f"quiz: {quiz}")
            result = self.parse_json_like(quiz)
            print(f"Result length: {len(result)}")
            print(f"Result: {result}")
            results.extend(result)

        end = time.time()
        random.shuffle(results)
        results = results[:self.number_of_questions]

        with open(f'{self.question_type}.json', 'w', encoding='utf-8') as f:
            json.dump({"Total time taken": end - start}, f)
            json.dump(results, f, indent=2)

        return results

class MCQ(QuizGenerator):
    """
    Concrete implementation of QuizGenerator for generating multiple-choice questions.
    """

    def __init__(self, chunk_size: int, chunk_overlap: int, llm: Any, number_of_questions: int, difficulty: str):
        super().__init__(chunk_size, chunk_overlap, llm, number_of_questions, difficulty, 'mcq')

    def generate_template(self) -> str:
        """
        Generate a template for creating multiple-choice questions.
        """
        template = f"""<s>[INST]Multiple-choice questions (MCQs) are a type of objective assessment item that presents respondents with a question or statement followed by a set of predefined answer options. The respondent must choose the correct answer from the provided options. Each question typically has only one correct answer.
                    You are an expert at creating scenario-based Multiple Choice Questions and answers, based on the given text and documentation.
                    Your goal is to make users acquainted with the content of the text through well-crafted Multiple Choice questions you generate.
                    IMPORTANT: Also, for each question, give the correct answer and don't use incomplete sentences as context.
                    IMPORTANT: Do not repeat the same question. Do not rephrase/jumble the same question multiple times.
                    You should also make sure to include the question and solution in one single post and not split them apart.
                    Make sure not to lose any important information.
                    Set the difficulty of the questions to {self.difficulty}. Make sure all questions are set to the specified difficulty.

                    The input text given is:

                    ------------
                    {{text}}
                    ------------

                    IMPORTANT: Create exactly {self.number_of_questions_per_chunk} questions and return the correct answer.
                    IMPORTANT: Do not create more than {self.number_of_questions_per_chunk} questions or less than {self.number_of_questions_per_chunk} questions.
                    IMPORTANT: For each question, give four options with one of them being the correct answer and the other options being incorrect (e.g., a) Incorrect, b) Incorrect, c) Incorrect, d) Correct, answer: d) Correct).
                    IMPORTANT: Make sure to follow the same format {{format_instructions}} for all questions.
                    Think clearly and follow the instructions tagged IMPORTANT with utmost criticality.
                    [/INST]</s>"""
        return template

    def generate_template_and_format(self) -> Tuple[str, str]:
        """
        Generate a template and format instructions for the LLM prompt.
        """
        template = self.generate_template()
        response_schema = [
            ResponseSchema(name="question", description="A multiple choice question generated from input text snippet."),
            ResponseSchema(name="a)", description="First option for the multiple choice question."),
            ResponseSchema(name="b)", description="Second option for the multiple choice question."),
            ResponseSchema(name="c)", description="Third option for the multiple choice question."),
            ResponseSchema(name="d)", description="Fourth option for the multiple choice question."),
            ResponseSchema(name="answer", description="Correct answer for the question. Use this format: 'd) the correct answer', etc.")
        ]

        output_parser = StructuredOutputParser.from_response_schemas(response_schema)
        format_instructions = output_parser.get_format_instructions()

        return template, format_instructions


class TrueFalse(QuizGenerator):
    def __init__(self, chunk_size: int, chunk_overlap: int, llm, number_of_questions: int, difficulty: str):
        """
        Initialize the TrueFalse class.

        Args:
            chunk_size (int): Size of the chunks for text splitting.
            chunk_overlap (int): Overlap size between chunks.
            llm (object): The large language model to be used.
            number_of_questions (int): Number of questions to generate.
            difficulty (str): Difficulty level of the questions.
        """
        super().__init__(chunk_size, chunk_overlap, llm, number_of_questions, difficulty, 'trueFalse')

    def generate_template(self) -> str:
        """
        Generate the prompt template for True/False questions.

        Returns:
            str: The prompt template string.
        """
        template = f""" <s>[INST]True/false questions are a type of binary or dichotomous closed-ended question format. In a true/false question, respondents are presented with a statement, and they are required to determine whether the statement is true or false. This format simplifies the response choices to two options, making it straightforward
                    You are an expert at creating scenario based True or False question and answers, based on the given text and documentation.
                    IMPORTANT: Your goal is to make users acquainted with the content of the text through well-crafted True/False questions you generate.
                    Also, for each question, give the correct answer and don't use incomplete sentences as context.
                    IMPORTANT: Do not repeat the same question. Do not rephrase/jumble the same question multiple times.
                    You should also make sure to include the question and solution in one single post and not split them apart.
                    Make sure not to lose any important information.
                    Set the difficulty of the questions to {self.difficulty}. Make sure all questions are set to the specified difficulty.

                    The input text given is:

                    ------------
                    {{text}}
                    ------------

                    Create {self.number_of_questions_per_chunk} questions and return the correct answer.
                    IMPORTANT: For each question, the correct answer must be binary, that is, either True or False only.
                    Make sure to follow the same format {{format_instructions}} for all questions.
                    Think clearly and follow the instructions tagged IMPORTANT with utmost criticality.
                    [/INST]</s>"""
        return template

    def generate_template_and_format(self) -> tuple:
        """
        Generate the prompt template and format instructions for True/False questions.

        Returns:
            tuple: A tuple containing the prompt template and format instructions.
        """
        template = self.generate_template()
        response_schema = [
            ResponseSchema(name="question", description="A True/False question generated from input text snippet."),
            ResponseSchema(name="answer", description="Correct answer for the question which is either 'True' or 'False' only.")
        ]

        output_parser = StructuredOutputParser.from_response_schemas(response_schema)
        format_instructions = output_parser.get_format_instructions()

        return template, format_instructions



class OpenEnded(QuizGenerator):
    def __init__(self, chunk_size: int, chunk_overlap: int, llm, number_of_questions: int, difficulty: str):
        """
        Initialize the OpenEnded class.

        Args:
            chunk_size (int): Size of the chunks for text splitting.
            chunk_overlap (int): Overlap size between chunks.
            llm (object): The large language model to be used.
            number_of_questions (int): Number of questions to generate.
            difficulty (str): Difficulty level of the questions.
        """
        super().__init__(chunk_size, chunk_overlap, llm, number_of_questions, difficulty, 'openEnded')

    def generate_template(self) -> str:
        """
        Generate the prompt template for open-ended questions.

        Returns:
            str: The prompt template string.
        """
        template = f""" <s>[INST]Open-ended questions are questions that cannot be answered with a simple 'yes' or 'no' or with a brief piece of information. Instead, they require a more elaborate and detailed response, encouraging the respondent to express their thoughts, opinions, or knowledge on a given topic.
                    You are an expert at creating scenario based Open-ended Question and answers, based on the given text and documentation.
                    Your goal is to make users acquainted with the content of the text through well-crafted Open-ended questions you generate.
                    IMPORTANT: Also, for each question, give the correct answer and don't use incomplete sentences as context.
                    IMPORTANT: Do not repeat the same question. Do not rephrase/jumble the same question multiple times.
                    You should also make sure to include the question and solution in one single post and not split them apart.
                    Make sure not to lose any important information.
                    Set the difficulty of the questions to {self.difficulty}. Make sure all questions are set to the specified difficulty.

                    The input text given is:

                    ------------
                    {{text}}
                    ------------

                    Create {self.number_of_questions_per_chunk} questions and return the correct answer.
                    IMPORTANT: Generate questions which have to be answered with at least half a dozen words along with a clear, descriptive answer.
                    Make sure to follow the same format {{format_instructions}} for all questions.
                    Think clearly and follow the instructions tagged IMPORTANT with utmost criticality.
                    [/INST]</s>"""
        return template

    def generate_template_and_format(self) -> tuple:
        """
        Generate the prompt template and format instructions for open-ended questions.

        Returns:
            tuple: A tuple containing the prompt template and format instructions.
        """
        template = self.generate_template()
        response_schema = [
            ResponseSchema(name="question", description="An open-ended question generated from input text snippet."),
            ResponseSchema(name="answer", description="Detailed answer for the open-ended question.")
        ]

        output_parser = StructuredOutputParser.from_response_schemas(response_schema)
        format_instructions = output_parser.get_format_instructions()

        return template, format_instructions



class FillInTheBlanks(QuizGenerator):
    def __init__(self, chunk_size: int, chunk_overlap: int, llm, number_of_questions: int, difficulty: str):
        """
        Initialize the FillInTheBlanks class.

        Args:
            chunk_size (int): Size of the chunks for text splitting.
            chunk_overlap (int): Overlap size between chunks.
            llm (object): The large language model to be used.
            number_of_questions (int): Number of questions to generate.
            difficulty (str): Difficulty level of the questions.
        """
        super().__init__(chunk_size, chunk_overlap, llm, number_of_questions, difficulty, "fillInTheBlanks")

    def generate_template(self) -> str:
        """
        Generate the prompt template for fill-in-the-blanks questions.

        Returns:
            str: The prompt template string.
        """
        template = f""" <s>[INST]Fill in the blanks questions, also known as completion questions, are a type of assessment or educational exercise where individuals are provided with a sentence, paragraph, or passage with one or more missing words or phrases. The task is to fill in the missing elements to complete the text.
                    You are an expert at creating scenario based Fill in the blanks questions and answers, based on the given text and documentation.
                    Your goal is to make users acquainted with the content of the text through well-crafted Fill in the blanks questions you generate.
                    IMPORTANT: Also, for each question, give the correct answer and don't use incomplete sentences as context.
                    IMPORTANT: Do not repeat the same question. Do not rephrase/jumble the same question multiple times.
                    You should also make sure to include the question and solution in one single post and not split them apart.
                    Make sure not to lose any important information.
                    Set the difficulty of the questions to {self.difficulty}. Make sure all questions are set to the specified difficulty.

                    The input text given is:

                    ------------
                    {{text}}
                    ------------

                    Create {self.number_of_questions_per_chunk} questions and return the correct answer.
                    IMPORTANT: Generate statements with one keyword replaced by a '_____' which makes up the question. The answer is then the replaced keyword.
                    Make sure to follow the same format {{format_instructions}} for all questions.
                    Think clearly and follow the instructions tagged IMPORTANT with utmost criticality.
                    [/INST]</s>"""
        return template

    def generate_template_and_format(self) -> tuple:
        """
        Generate the prompt template and format instructions for fill-in-the-blanks questions.

        Returns:
            tuple: A tuple containing the prompt template and format instructions.
        """
        template = self.generate_template()
        response_schema = [
            ResponseSchema(name="question", description="A fill-in-the-blanks question with the keyword represented by a blank, generated from input text snippet."),
            ResponseSchema(name="answer", description="Correct answer for the question. Provide the answer with blanks filled.")
        ]

        output_parser = StructuredOutputParser.from_response_schemas(response_schema)
        format_instructions = output_parser.get_format_instructions()

        return template, format_instructions


class Mixed(QuizGenerator):
    def __init__(self, chunk_size: int, chunk_overlap: int, llm, number_of_questions: int, difficulty: str):
        """
        Initialize the Mixed class.

        Args:
            chunk_size (int): Size of the chunks for text splitting.
            chunk_overlap (int): Overlap size between chunks.
            llm (object): The large language model to be used.
            number_of_questions (int): Number of questions to generate.
            difficulty (str): Difficulty level of the questions.
        """
        super().__init__(chunk_size, chunk_overlap, llm, number_of_questions, difficulty, "mixed")
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.llm = llm
        self.number_of_questions = number_of_questions
        self.difficulty = difficulty

    def combined_questions(self, mcq, trueFalse, openEnded, text: str) -> list:
        """
        Generate combined questions from multiple types (MCQ, True/False, Open-Ended).

        Args:
            mcq (MCQ): An instance of the MCQ class.
            trueFalse (TrueFalse): An instance of the TrueFalse class.
            openEnded (OpenEnded): An instance of the OpenEnded class.
            text (str): The input text to generate questions from.

        Returns:
            list: A list of combined questions.
        """
        mcqs = mcq.generate_qa(text=text)
        tfs = trueFalse.generate_qa(text=text)
        ops = openEnded.generate_qa(text=text)
        return mcqs + ops + tfs

    def generate_qa(self, file_path: str = None, text: str = None) -> list:
        """
        Generate mixed questions from file or text input.

        Args:
            file_path (str, optional): Path to the input file. Defaults to None.
            text (str, optional): Input text data. Defaults to None.

        Returns:
            list: A list of generated questions.
        """
        if file_path and text:
            print("Input either a file or text data. Not both.")
            return []

        elif file_path:
            chunks = self.file_processing(file_path)

        elif text:
            chunks = self.text_splitter(text)

        else:
            print("Please provide either a file path or text data.")
            return []

        self.number_of_questions_per_chunk = math.ceil(self.number_of_questions / (len(chunks) * 3))
        mcq = MCQ(self.chunk_size, self.chunk_overlap, self.llm, self.number_of_questions_per_chunk, self.difficulty)
        trueFalse = TrueFalse(self.chunk_size, self.chunk_overlap, self.llm, self.number_of_questions_per_chunk, self.difficulty)
        openEnded = OpenEnded(self.chunk_size, self.chunk_overlap, self.llm, self.number_of_questions_per_chunk, self.difficulty)

        start = time.time()
        results = []
        for chunk in chunks:
            result = self.combined_questions(mcq, trueFalse, openEnded, chunk)
            print("result len: ", len(result))
            print(f"result: {result}")
            results.extend(result)
        end = time.time()
        random.shuffle(results)
        results = results[:self.number_of_questions]
        with open('mixed.json', 'w', encoding='utf-8') as f:
            json.dump(f"Total time taken: {end-start}", f)
            json.dump(results, f, indent=2)
        print(end - start)
        return results



## Run

In [ ]:
# MCQ
mcq = MCQ(5000, 50, llm, 5, "easy")
mcq_res = mcq.generate_qa(text=text)

Length of chunks: 4
Template: input_variables=['format_instructions', 'text'] template="<s>[INST]Multiple-choice questions (MCQs) are a type of objective assessment item that presents respondents with a question or statement followed by a set of predefined answer options. The respondent must choose the correct answer from the provided options. Each question typically has only one correct answer.\n                    You are an expert at creating scenario-based Multiple Choice Questions and answers, based on the given text and documentation.\n                    Your goal is to make users acquainted with the content of the text through well-crafted Multiple Choice questions you generate.\n                    IMPORTANT: Also, for each question, give the correct answer and don't use incomplete sentences as context.\n                    IMPORTANT: Do not repeat the same question. Do not rephrase/jumble the same question multiple times.\n                    You should also make sure to incl

In [ ]:
# True or False
trueFalse = TrueFalse(5000, 50, llm, 5, "easy")
tf_res = trueFalse.generate_qa(file_path='./input.txt')

Length of chunks: 4
Template: input_variables=['format_instructions', 'text'] template=" <s>[INST]True/false questions are a type of binary or dichotomous closed-ended question format. In a true/false question, respondents are presented with a statement, and they are required to determine whether the statement is true or false. This format simplifies the response choices to two options, making it straightforward\n                    You are an expert at creating scenario based True or False question and answers, based on the given text and documentation.\n                    IMPORTANT: Your goal is to make users acquainted with the content of the text through well-crafted True/False questions you generate.\n                    Also, for each question, give the correct answer and don't use incomplete sentences as context.\n                    IMPORTANT: Do not repeat the same question. Do not rephrase/jumble the same question multiple times.\n                    You should also make sur

In [20]:
#Open-Ended
openEnded = OpenEnded(5000, 50, llm, 5, "easy")
oe_res = openEnded.generate_qa(text=text)

Length of chunks: 4
Template: input_variables=['format_instructions', 'text'] template=" <s>[INST]Open-ended questions are questions that cannot be answered with a simple 'yes' or 'no' or with a brief piece of information. Instead, they require a more elaborate and detailed response, encouraging the respondent to express their thoughts, opinions, or knowledge on a given topic.\n                    You are an expert at creating scenario based Open-ended Question and answers, based on the given text and documentation.\n                    Your goal is to make users acquainted with the content of the text through well-crafted Open-ended questions you generate.\n                    IMPORTANT: Also, for each question, give the correct answer and don't use incomplete sentences as context.\n                    IMPORTANT: Do not repeat the same question. Do not rephrase/jumble the same question multiple times.\n                    You should also make sure to include the question and solution 

In [1]:
#Mixed
mixed = Mixed(5000, 50, llm, 10, "easy")
mixed_res = mixed.generate_qa(text=text)

## Quiz Interface with Streamlit

In [ ]:
import streamlit as st
from sentence_transformers import SentenceTransformer, util


def openEnded_score(self, ground_truth, user_answer):
        """
        Calculate the regression score using semantic similarity between original and RAG answers.

        Returns:
            list: List of similarity scores for each question.
        """
        model = SentenceTransformer('all-MiniLM-L6-v2')

        # Texts to compare

        # Compute embeddings and similarity
        for i in range(len(ground_truth)):
            original_embedding = model.encode(ground_truth)
            rag_embedding = model.encode(user_answer)

            # Compute similarity
            similarity = util.pytorch_cos_sim(original_embedding, rag_embedding)
            print(f"Similarity for question {i+1}: {similarity.item()}")


        return similarity

# Initialize or retrieve the session state
def quiz_interface(questions):

  if 'answers' not in st.session_state:
    st.session_state.answers = {}

  # Display the questions
  st.title("ReiviseMate")

  for i, q in enumerate(questions):
      st.write(f"Q{i+1}: {q['question']}")
      if q["type"] == "MCQ":
          answer = st.radio(f"Choose an answer for Q{i+1}", q["options"], key=f"q{i}")
      elif q["type"] == "True/False":
          options = ["True", "False"]
          answer = st.radio(f"Choose an answer for Q{i+1}", options, key=f"q{i}")
      elif q["type"] == "Open-ended":
          answer = st.text_area(f"Your answer for Q{i+1}", key=f"q{i}")

      # Save the answer to session state
      st.session_state.answers[f"q{i}"] = answer

  # Submit button to calculate score
  if st.button("Submit Quiz"):
      score = 0
      total_questions = len([q for q in questions])

      for i, q in enumerate(questions):
          if q['type'] != 'Open-ended' and st.session_state.answers[f"q{i}"] == q['answer']:
              score += 1

          if q['type'] == 'Open-ended':
              score += openEnded_score(f"q{i}", q['answer'])

      st.write(f"Your score: {score}/{total_questions}")

      # Display correct answers for review
      for i, q in enumerate(questions):
          st.write(f"Q{i+1}: {q['question']}")
          st.write(f"Your answer: {st.session_state.answers[f'q{i}']}")
          st.write(f"Correct answer: {q['answer']}")
          st.write("")

In [ ]:
quiz_interface(mcq_res+trueFalse_res+openEnded_res)

## RegressionTest class

In [13]:
class RegressionTest(ABC):

    def __init__(self, chunk_size, chunk_overlap, llm, results):
        """
        Initialize the RegressionTest class.

        Args:
            chunk_size (int): Size of the chunks for text splitting.
            chunk_overlap (int): Overlap size between chunks.
            llm (object): The large language model to be used.
            results (list): List of original answers for comparison.
        """
        self.retriever = self.get_retriever(chunk_size, chunk_overlap)
        self.llm = llm
        self.questions = results

    def get_retriever(self, chunk_size, chunk_overlap):
        """
        Set up the document retriever.

        Args:
            chunk_size (int): Size of the chunks for text splitting.
            chunk_overlap (int): Overlap size between chunks.

        Returns:
            retriever (object): The retriever object for document retrieval.
        """
        # Load documents from the input file
        loader = TextLoader("./input.txt")
        docs = loader.load()

        # Create embeddings using HuggingFace model
        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

        # Split documents into chunks
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        documents = text_splitter.split_documents(docs)
        print(f"documents: {documents}")

        # Create a vector store using Chroma
        try:
            db = Chroma.from_documents(documents, embeddings)
        except InvalidDimensionException:
            Chroma().delete_collection()
            db = Chroma.from_documents(documents, embeddings)

        # Return retriever object
        return db.as_retriever(search_kwargs={"k": 2})

    @abstractmethod
    def get_template(self):
        """
        Abstract method to get the template for the prompt.
        """
        pass

    @abstractmethod
    def get_rag_answers(self):
        """
        Abstract method to get the RAG answers.
        """
        pass

    @abstractmethod
    def get_original_answers(self):
        """
        Abstract method to get the original answers.
        """
        pass

    def format_docs(self, docs):
        """
        Format documents into a single string.

        Args:
            docs (list): List of document objects.

        Returns:
            str: Formatted string of document contents.
        """
        return "\n\n".join(doc.page_content for doc in docs)

    def dict_to_multiline_string(self, dictionary):
        """
        Convert a dictionary to a multi-line string.

        Args:
            dictionary (dict): Dictionary to be converted.

        Returns:
            str: Multi-line string representation of the dictionary.
        """
        lines = []
        for key, value in dictionary.items():
            if key != "answer":
                line = f"{key}: {value}"
                lines.append(line)
        return "\n".join(lines)

    def get_rag_chain(self):
        """
        Set up the RAG (Retrieval-Augmented Generation) chain.

        Returns:
            rag_chain (RunnableChain): The RAG chain object.
        """
        # Get the prompt template
        template = self.get_template()
        prompt = PromptTemplate(
            input_variables=["context", "question"],
            template=template,
        )

        # Create the LLM chain with the given prompt
        llm_chain = LLMChain(llm=self.llm, prompt=prompt)

        # Set up the RAG chain
        rag_chain = (
            {"context": self.retriever | self.format_docs, "question": RunnablePassthrough()}
            | llm_chain
        )

        return rag_chain

    def regression_score(self):
        """
        Calculate the regression score by comparing RAG and original answers.

        Returns:
            float: The accuracy score in percentage.
        """
        # Get the RAG and original answers
        rag_results = self.get_rag_answers()
        original_results = self.get_original_answers()

        # Print debugging information
        print(f"original_results: {original_results}")
        print(f"rag_results: {rag_results}")

        # Calculate the matching count and accuracy
        matching_count = sum(1 for x, y in zip(original_results, rag_results) if x == y)
        accuracy = matching_count / len(original_results)

        # Print and return the accuracy
        accuracy_percentage = round(accuracy * 100, 2)
        print(accuracy_percentage)
        return accuracy_percentage


## RegressionTest subclasses for different question types

### Multiple Choice questions

In [16]:
import re
from abc import ABC

class MCQTest(RegressionTest):
    def __init__(self, chunk_size, chunk_overlap, llm, results):
        """
        Initialize the MCQTest class by inheriting from RegressionTest.

        Args:
            chunk_size (int): Size of the chunks for text splitting.
            chunk_overlap (int): Overlap size between chunks.
            llm (object): The large language model to be used.
            results (list): List of original questions and answers for comparison.
        """
        super().__init__(chunk_size, chunk_overlap, llm, results)

    def get_original_answers(self):
        """
        Extract original answers from the questions.

        Returns:
            list: List of original answers in the format 'a)', 'b)', etc.
        """
        original_results = []
        for question in self.questions:
            answer = question['answer']
            pattern = r"[abcd]\)"
            match = re.search(pattern, answer)
            if match:
                original_results.append(match.group())

        return original_results

    def get_template(self):
        """
        Get the prompt template for the LLM to answer MCQs.

        Returns:
            str: The prompt template string.
        """
        prompt = """
        You are an expert at answering multiple choice questions.
        Use the following pieces of context to answer the multiple choice question.
        Choose and return the correct answer with the option head.
        Your response should always follow the format: 'b) something', where 'b)' - is the option head and 'something' is the answer.
        The option head should always be of the format - alphabet followed by ')'.
        Think carefully and read the context properly to answer the question.
        There is no need for any explanation.
        If you don't know the answer, just say that you don't know, don't try to make up an answer.
        {context}
        Question: {question}
        """

        return prompt

    def get_rag_answers(self):
        """
        Get answers generated by the RAG (Retrieval-Augmented Generation) model.

        Returns:
            list: List of generated answers in the format 'a)', 'b)', etc.
        """
        results = []
        for question in self.questions:
            # Convert question dictionary to multi-line string
            question_text = self.dict_to_multiline_string(question)
            print(f"Question: {question_text}")

            # Get the RAG chain for processing
            rag_chain = self.get_rag_chain()
            result = rag_chain.invoke(question_text)
            print(f"Result: {result}")

            # Extract the answer option from the generated text
            pattern = r"[abcd]\)"
            match = re.search(pattern, result['text'])
            if match:
                results.append(match.group())

        return results


In [33]:
mcq_test = MCQTest(5000, 50, llm_google, mcq_res)
mcq_test.regression_score()

documents: [Document(metadata={'source': './input.txt'}, page_content='text = """\nBarack Obama\nOfficial portrait, 2012\n44th President of the United States\nIn office\nJanuary 20, 2009 – January 20, 2017\nVice PresidentJoe Biden\nPreceded by George W. Bush\nSucceeded byDonald Trump\nUnited States Senator\nfrom Illinois\nIn office\nJanuary 3, 2005 – November 16, 2008\nPreceded by Peter Fitzgerald\nSucceeded byRoland Burris\nMember of the Illinois Senate\nfrom the 13th district\nIn office\nJanuary 8, 1997 – November 4, 2004\nPreceded by Alice Palmer\nSucceeded byKwame Raoul\nPersonal detailsBarack Obama\nBarack Hussein Obama II (/b ə ˈr ɑ ːk hu ː ˈse ɪn\no ʊ ˈb ɑ ːm ə/ ⓘ b ə-RAHK hoo- SAYN oh-BAH-m ə;[1]\nborn Augus t 4, 1961)  is an American politician who\nserved as the 44th president of the United States from\n2009 to 2017. A member of the Democratic Party, he was\nthe first African-American president in U.S. history.\nObama previously served as a U.S. senator representing\nIllinois

40.0

### True or False questions

In [ ]:
import re
from abc import ABC

class TrueFalseTest(RegressionTest):
    def __init__(self, chunk_size, chunk_overlap, llm, results):
        """
        Initialize the TrueFalseTest class by inheriting from RegressionTest.

        Args:
            chunk_size (int): Size of the chunks for text splitting.
            chunk_overlap (int): Overlap size between chunks.
            text (str): The input text to be used for generating context.
            llm (object): The large language model to be used.
            results (list): List of original questions and answers for comparison.
        """
        super().__init__(chunk_size, chunk_overlap, llm, results)

    def get_template(self):
        """
        Get the prompt template for the LLM to answer True/False questions.

        Returns:
            str: The prompt template string.
        """
        template = """<s>[INST]
        You are an expert at answering True or False questions.
        Use the following pieces of context to answer the True or False question.
        You are to answer the question only from the provided context. Do not use anything else as a source of information.
        The only possible answers are true or false.
        Analyze the question carefully and return the correct answer.
        Your response should always be either true or false only.
        There is no need for any explanation.
        If you don't know the answer, just say that you don't know, don't try to make up an answer.[/INST]
        {context}
        [INST] Question: {question} [/INST]
        </s>"""

        return template

    def get_original_answers(self):
        """
        Extract original answers from the questions.

        Returns:
            list: List of original answers in lowercase ('true' or 'false').
        """
        original_results = []
        for question in self.questions:
            answer = question['answer']
            original_results.append(answer.lower())
        return original_results

    def get_rag_answers(self):
        """
        Get answers generated by the RAG (Retrieval-Augmented Generation) model.

        Returns:
            list: List of generated answers in lowercase ('true' or 'false').
        """
        results = []
        for question in self.questions:
            # Convert question dictionary to multi-line string
            question_text = self.dict_to_multiline_string(question)
            print(f"Question: {question_text}")

            # Get the RAG chain for processing
            rag_chain = self.get_rag_chain()
            result = rag_chain.invoke(question_text)
            print(f"Result: {result}")

            # Extract the 'true' or 'false' answer from the generated text
            pattern = r"\b(true|false)\b"
            match = re.findall(pattern, result['text'], re.IGNORECASE)
            if match:
                results.append(match[0].lower())

        return results


In [ ]:
trueFalse_test = TrueFalseTest(5000, 50, text, llm_google, tf_res)
trueFalse_test.regression_score()

documents: [Document(page_content='text = """\nBarack Obama\nOfficial portrait, 2012\n44th President of the United States\nIn office\nJanuary 20, 2009 – January 20, 2017\nVice PresidentJoe Biden\nPreceded by George W. Bush\nSucceeded byDonald Trump\nUnited States Senator\nfrom Illinois\nIn office\nJanuary 3, 2005 – November 16, 2008\nPreceded by Peter Fitzgerald\nSucceeded byRoland Burris\nMember of the Illinois Senate\nfrom the 13th district\nIn office\nJanuary 8, 1997 – November 4, 2004\nPreceded by Alice Palmer\nSucceeded byKwame Raoul\nPersonal detailsBarack Obama\nBarack Hussein Obama II (/b ə ˈr ɑ ːk hu ː ˈse ɪn\no ʊ ˈb ɑ ːm ə/ ⓘ b ə-RAHK hoo- SAYN oh-BAH-m ə;[1]\nborn Augus t 4, 1961)  is an American politician who\nserved as the 44th president of the United States from\n2009 to 2017. A member of the Democratic Party, he was\nthe first African-American president in U.S. history.\nObama previously served as a U.S. senator representing\nIllinois from 2005 to 2008, as an Illinois s

100.0

### Open-ended questions

In [31]:
from sentence_transformers import SentenceTransformer, util

class OpenEndedTest(RegressionTest):

    def __init__(self, chunk_size, chunk_overlap, llm, results):
        """
        Initialize the OpenEndedTest class by inheriting from RegressionTest.

        Args:
            chunk_size (int): Size of the chunks for text splitting.
            chunk_overlap (int): Overlap size between chunks.
            text (str): The input text to be used for generating context.
            llm (object): The large language model to be used.
            results (list): List of original questions and answers for comparison.
        """
        super().__init__(chunk_size, chunk_overlap, llm, results)

    def get_template(self):
        """
        Get the prompt template for the LLM to answer open-ended questions.

        Returns:
            str: The prompt template string.
        """
        template = """[INST]
        You are an expert at answering Open-ended questions.
        Use the following pieces of context to answer the open-ended questions.
        Keep the answer brief and precise.
        If you don't know the answer, just say that you don't know, don't try to make up an answer.
        {context}
        Question: {question}
        [/INST]
        """
        return template

    def get_original_answers(self):
        """
        Extract original answers from the questions.

        Returns:
            list: List of original answers.
        """
        original_results = []
        for question in self.questions:
            answer = question['answer']
            original_results.append(answer)
        return original_results

    def get_rag_answers(self):
        """
        Get answers generated by the RAG (Retrieval-Augmented Generation) model.

        Returns:
            list: List of generated answers.
        """
        results = []
        for question in self.questions:
            # Convert question dictionary to multi-line string
            question_text = self.dict_to_multiline_string(question)
            print(f"Question: {question_text}")

            # Get the RAG chain for processing
            rag_chain = self.get_rag_chain()
            result = rag_chain.invoke(question_text)
            print(f"Result: {result}")

            # Clean and store the result
            clean_result = result['text'].strip().replace("\n", "")
            results.append(clean_result)

        return results

    def regression_score(self):
        """
        Calculate the regression score using semantic similarity between original and RAG answers.

        Returns:
            list: List of similarity scores for each question.
        """
        model = SentenceTransformer('all-MiniLM-L6-v2')

        # Texts to compare
        original_results = self.get_original_answers()
        rag_results = self.get_rag_answers()
        similarity_scores = []

        # Compute embeddings and similarity
        for i in range(len(original_results)):
            original_embedding = model.encode(original_results[i])
            rag_embedding = model.encode(rag_results[i])

            # Compute similarity
            similarity = util.pytorch_cos_sim(original_embedding, rag_embedding)
            print(f"Similarity for question {i+1}: {similarity.item()}")
            similarity_scores.append(similarity.item())

        return similarity_scores


In [32]:
openEnded_test = OpenEndedTest(5000, 50, llm_google, oe_res)
openEnded_test.regression_score()

documents: [Document(metadata={'source': './input.txt'}, page_content='text = """\nBarack Obama\nOfficial portrait, 2012\n44th President of the United States\nIn office\nJanuary 20, 2009 – January 20, 2017\nVice PresidentJoe Biden\nPreceded by George W. Bush\nSucceeded byDonald Trump\nUnited States Senator\nfrom Illinois\nIn office\nJanuary 3, 2005 – November 16, 2008\nPreceded by Peter Fitzgerald\nSucceeded byRoland Burris\nMember of the Illinois Senate\nfrom the 13th district\nIn office\nJanuary 8, 1997 – November 4, 2004\nPreceded by Alice Palmer\nSucceeded byKwame Raoul\nPersonal detailsBarack Obama\nBarack Hussein Obama II (/b ə ˈr ɑ ːk hu ː ˈse ɪn\no ʊ ˈb ɑ ːm ə/ ⓘ b ə-RAHK hoo- SAYN oh-BAH-m ə;[1]\nborn Augus t 4, 1961)  is an American politician who\nserved as the 44th president of the United States from\n2009 to 2017. A member of the Democratic Party, he was\nthe first African-American president in U.S. history.\nObama previously served as a U.S. senator representing\nIllinois

[0.5447726249694824,
 0.5447726249694824,
 0.5447726249694824,
 0.5447726249694824,
 0.6848662495613098]

## API endpoints using FastAPI

In [ ]:
from fastapi import FastAPI, Request
from langchain_community.llms import CTransformers
import locale

# Ensure the locale encoding is set to UTF-8
locale.getpreferredencoding = lambda: "UTF-8"

# Initialize the FastAPI app
app = FastAPI()

# Configuration for the LLM
config = {
    'max_new_tokens': 50000,
    'repetition_penalty': 1.1,
    'context_length': 10000,
    'temperature': 0,
    'gpu_layers': 100,
}

# Path to the local LLM model
llm_model_path = "/home/ubuntu/mistral/openhermes-2.5-mistral-7b-16k.Q4_K_M.gguf"

# Instantiate the LLM with the given configuration
llm = CTransformers(
    model=llm_model_path,
    model_type="mistral",
    gpu_layers=config['gpu_layers'],
    config=config,
    mlock=True
)

@app.get("/status")
async def status(request: Request):
    """
    Endpoint to check the status of the API.

    Args:
        request (Request): The request object.

    Returns:
        dict: A dictionary indicating the status of the API.
    """
    return {"status": "running"}

@app.post("/api/generate")
async def llm_chain(request: Request):
    """
    Endpoint to generate a response from the LLM based on the given prompt.

    Args:
        request (Request): The request object containing the prompt.

    Returns:
        dict: A dictionary containing the generated result.
    """
    request_body = await request.json()
    prompt = request_body['prompt']
    result = llm.invoke(prompt)
    return {"result": str(result)}

if __name__ == "__main__":
    import uvicorn
    # Run the FastAPI app using Uvicorn
    uvicorn.run(app, host="127.0.0.1", port=18889)
